# Buổi 2 – Đánh giá RAG với ragas

Đánh giá quy trình RAG tối giản bằng các chỉ số của ragas: answer_relevancy, faithfulness, context_precision.


# Kịch bản
Kịch bản này đánh giá một pipeline Retrieval Augmented Generation (RAG) tối giản tại địa phương. Chúng ta:
- Định nghĩa một tập hợp tài liệu tổng hợp nhỏ.
- Gắn kết tài liệu & triển khai một bộ truy xuất tương tự đơn giản.
- Tạo câu trả lời có căn cứ sử dụng một mô hình cục bộ (Foundry Local / tương thích OpenAI).
- Tính toán các chỉ số ragas (`answer_relevancy`, `faithfulness`, `context_precision`).
- Hỗ trợ chế độ NHANH (biến môi trường `RAG_FAST=1`) để chỉ tính toán mức độ liên quan của câu trả lời nhằm lặp lại nhanh chóng.

Sử dụng notebook này để xác minh rằng mô hình cục bộ + ngăn xếp nhúng của bạn tạo ra các câu trả lời có căn cứ thực tế trước khi mở rộng sang các tập hợp tài liệu lớn hơn.


### Giải thích: Cài đặt phụ thuộc
Cài đặt các thư viện cần thiết:
- `foundry-local-sdk` để quản lý mô hình cục bộ.
- Giao diện khách hàng `openai`.
- `sentence-transformers` để tạo các embedding dày đặc.
- `ragas` + `datasets` để đánh giá và tính toán các chỉ số.
- Bộ chuyển đổi `langchain-openai` cho giao diện LLM của ragas.

Có thể chạy lại an toàn; bỏ qua nếu môi trường đã được chuẩn bị.


In [1]:
# Install libraries (ragas pulls datasets, evaluate, etc.)
!pip install -q foundry-local-sdk openai sentence-transformers ragas datasets numpy langchain-openai

### Giải thích: Nhập lõi & Các chỉ số
Tải các thư viện lõi và các chỉ số ragas. Các thành phần chính:
- SentenceTransformer để tạo embeddings.
- `evaluate` + các chỉ số ragas được chọn.
- `Dataset` để xây dựng tập dữ liệu đánh giá.
Những lần nhập này không kích hoạt các cuộc gọi từ xa (ngoại trừ khả năng tải bộ nhớ đệm mô hình cho embeddings).


In [2]:
import os, numpy as np
from sentence_transformers import SentenceTransformer
from foundry_local import FoundryLocalManager
from openai import OpenAI
from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness, context_precision
from datasets import Dataset

### Giải thích: Tập dữ liệu nhỏ & Đáp án chuẩn cho câu hỏi
Định nghĩa một tập dữ liệu nhỏ trong bộ nhớ (`DOCS`), một tập hợp các câu hỏi của người dùng và các đáp án chuẩn mong đợi. Những điều này cho phép tính toán các chỉ số nhanh chóng và có tính xác định mà không cần truy xuất dữ liệu bên ngoài. Trong các tình huống thực tế, bạn sẽ lấy mẫu các truy vấn sản xuất + các đáp án được chọn lọc.


In [3]:
DOCS = [
 'Foundry Local exposes a local OpenAI-compatible endpoint.',
 'RAG retrieves relevant context snippets before generation.',
 'Local inference improves privacy and reduces latency.',
]
QUESTIONS = [
 'What advantage does local inference offer?',
 'How does RAG improve grounding?',
]
GROUND_TRUTH = [
 'It reduces latency and preserves privacy.',
 'It adds retrieved context snippets for factual grounding.',
]

### Giải thích: Khởi tạo dịch vụ, Embeddings & Bản vá an toàn
Khởi tạo trình quản lý Foundry Local, áp dụng bản vá an toàn chống lệch cấu trúc cho `promptTemplate`, xác định id mô hình, tạo client tương thích với OpenAI, và tính toán trước embeddings dày đặc cho tập hợp tài liệu. Điều này thiết lập trạng thái có thể tái sử dụng cho việc truy xuất + tạo nội dung.


In [4]:
import os
from foundry_local import FoundryLocalManager
from foundry_local.models import FoundryModelInfo
from openai import OpenAI

# --- Safe monkeypatch for potential null promptTemplate field (schema drift guard) ---
_original_from_list_response = FoundryModelInfo.from_list_response

def _safe_from_list_response(response):  # type: ignore
    try:
        if isinstance(response, dict) and response.get("promptTemplate") is None:
            response["promptTemplate"] = {}
    except Exception as e:  # pragma: no cover
        print(f"Warning normalizing promptTemplate: {e}")
    return _original_from_list_response(response)

if getattr(FoundryModelInfo.from_list_response, "__name__", "") != "_safe_from_list_response":
    FoundryModelInfo.from_list_response = staticmethod(_safe_from_list_response)  # type: ignore
# --- End monkeypatch ---

alias = os.getenv('FOUNDRY_LOCAL_ALIAS','phi-3.5-mini')
manager = FoundryLocalManager(alias)
print(f"Service running: {manager.is_service_running()} | Endpoint: {manager.endpoint}")
print('Cached models:', manager.list_cached_models())
model_info = manager.get_model_info(alias)
model_id = model_info.id
print(f"Using model id: {model_id}")

# OpenAI-compatible client
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key or 'not-needed')

from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
import numpy as np
doc_emb = embedder.encode(DOCS, convert_to_numpy=True, normalize_embeddings=True)


Service running: True | Endpoint: http://127.0.0.1:57127/v1
Cached models: [FoundryModelInfo(alias=gpt-oss-20b, id=gpt-oss-20b-cuda-gpu:1, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=9882 MB, license=apache-2.0), FoundryModelInfo(alias=phi-3.5-mini, id=Phi-3.5-mini-instruct-cuda-gpu:1, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=2181 MB, license=MIT), FoundryModelInfo(alias=phi-4-mini, id=Phi-4-mini-instruct-cuda-gpu:4, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=3686 MB, license=MIT), FoundryModelInfo(alias=qwen2.5-0.5b, id=qwen2.5-0.5b-instruct-cuda-gpu:3, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=528 MB, license=apache-2.0), FoundryModelInfo(alias=qwen2.5-7b, id=qwen2.5-7b-instruct-cuda-gpu:3, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=4843 MB, license=apache-2.0), FoundryModelInfo(alias=qwen2.5-coder-7b, id=qwen2.5-coder-7b-instruct-cuda-gpu:3, execution_p

c:\Users\leestott\AppData\Local\miniforge\envs\demo\lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


### Giải thích: Hàm Retriever  
Định nghĩa một trình truy xuất tương tự vector đơn giản sử dụng tích vô hướng trên các embedding đã được chuẩn hóa. Trả về các tài liệu top-k (mặc định k=2). Trong môi trường sản xuất, thay thế bằng chỉ mục ANN (FAISS, Chroma, Milvus) để tăng quy mô và giảm độ trễ.


In [5]:
def retrieve(query, k=2):
    q = embedder.encode([query], convert_to_numpy=True, normalize_embeddings=True)[0]
    sims = doc_emb @ q
    return [DOCS[i] for i in sims.argsort()[::-1][:k]]

### Giải thích: Hàm Tạo Dữ Liệu
`generate` tạo một lời nhắc bị giới hạn (hệ thống yêu cầu CHỈ sử dụng ngữ cảnh) và gọi mô hình cục bộ. Nhiệt độ thấp (0.1) ưu tiên việc trích xuất chính xác hơn là sáng tạo. Trả về văn bản câu trả lời đã được cắt gọn.


In [6]:
def generate(query, contexts):
    ctx = "\n".join(contexts)
    messages = [
        {'role':'system','content':'Answer using ONLY the provided context.'},
        {'role':'user','content':f"Context:\n{ctx}\n\nQuestion: {query}"}
    ]
    resp = client.chat.completions.create(model=model_id, messages=messages, max_tokens=120, temperature=0.1)
    return resp.choices[0].message.content.strip()


### Giải thích: Khởi tạo Fallback Client  
Đảm bảo `client` tồn tại ngay cả khi ô khởi tạo trước đó bị bỏ qua hoặc thất bại—ngăn chặn lỗi NameError trong các bước đánh giá sau.


In [7]:
# Fallback client initialization (added after patch failure)
try:
    client  # type: ignore
except NameError:
    from openai import OpenAI
    client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key or 'not-needed')
    print('Initialized OpenAI-compatible client (late init).')


### Giải thích: Vòng lặp đánh giá & Các chỉ số
Tạo tập dữ liệu đánh giá (các cột bắt buộc: question, answer, contexts, ground_truths, reference) sau đó lặp qua các chỉ số ragas đã chọn.

Tối ưu hóa:
- FAST_MODE giới hạn ở mức độ liên quan của câu trả lời để kiểm tra nhanh.
- Vòng lặp theo từng chỉ số tránh việc tính toán lại toàn bộ khi một chỉ số bị lỗi.

Xuất ra một dict của metric -> score (NaN nếu thất bại).


In [8]:
# Build evaluation dataset with required columns (including 'reference' for context_precision)
records = []
for q, gt in zip(QUESTIONS, GROUND_TRUTH):
    ctxs = retrieve(q)
    ans = generate(q, ctxs)
    records.append({
        'question': q,
        'answer': ans,
        'contexts': ctxs,
        'ground_truths': [gt],
        'reference': gt
    })

from datasets import Dataset
from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness, context_precision
from langchain_openai import ChatOpenAI
from ragas.run_config import RunConfig
import math, time, os
import numpy as np

ragas_llm = ChatOpenAI(model=model_id, base_url=manager.endpoint, api_key=manager.api_key or 'not-needed', temperature=0.0, timeout=60)

class LocalEmbeddings:
    def embed_documents(self, texts):
        return embedder.encode(texts, convert_to_numpy=True, normalize_embeddings=True).tolist()
    def embed_query(self, text):
        return embedder.encode([text], convert_to_numpy=True, normalize_embeddings=True)[0].tolist()

# Fast mode: only answer_relevancy unless RAG_FAST=0
FAST_MODE = os.getenv('RAG_FAST','1') == '1'
metrics = [answer_relevancy] if FAST_MODE else [answer_relevancy, faithfulness, context_precision]

base_timeout = 45 if FAST_MODE else 120

ds = Dataset.from_list(records)
print('Evaluation dataset columns:', ds.column_names)
print('Metrics to compute:', [m.name for m in metrics])

results_dict = {}
for metric in metrics:
    t0 = time.time()
    try:
        cfg = RunConfig(timeout=base_timeout, max_workers=1)
        partial = evaluate(ds, metrics=[metric], llm=ragas_llm, embeddings=LocalEmbeddings(), run_config=cfg, show_progress=False)
        raw_val = partial[metric.name]
        if isinstance(raw_val, list):
            numeric = [v for v in raw_val if isinstance(v, (int, float))]
            score = float(np.nanmean(numeric)) if numeric else math.nan
        else:
            score = float(raw_val)
        results_dict[metric.name] = score
    except Exception as e:
        results_dict[metric.name] = math.nan
        print(f"Metric {metric.name} failed: {e}")
    finally:
        print(f"{metric.name} finished in {time.time()-t0:.1f}s -> {results_dict[metric.name]}")

print('RAG evaluation results:', results_dict)
results_dict

Evaluation dataset columns: ['question', 'answer', 'contexts', 'ground_truths', 'reference']
Metrics to compute: ['answer_relevancy']


LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.


answer_relevancy finished in 78.1s -> 0.6975427764759168
RAG evaluation results: {'answer_relevancy': 0.6975427764759168}


{'answer_relevancy': 0.6975427764759168}


---

**Tuyên bố miễn trừ trách nhiệm**:  
Tài liệu này đã được dịch bằng dịch vụ dịch thuật AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mặc dù chúng tôi cố gắng đảm bảo độ chính xác, xin lưu ý rằng các bản dịch tự động có thể chứa lỗi hoặc không chính xác. Tài liệu gốc bằng ngôn ngữ bản địa nên được coi là nguồn thông tin chính thức. Đối với các thông tin quan trọng, nên sử dụng dịch vụ dịch thuật chuyên nghiệp của con người. Chúng tôi không chịu trách nhiệm cho bất kỳ sự hiểu lầm hoặc diễn giải sai nào phát sinh từ việc sử dụng bản dịch này.
